In [17]:
import mlflow
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType

# Set the tracking URI to your SQLite database
mlflow.set_tracking_uri("sqlite:///mlflow.db")

# Initialize the client
client = MlflowClient()

# List all experiments using the fluent API
experiments = mlflow.search_experiments(view_type=ViewType.ALL)

# Print the list of experiments
for experiment in experiments:
    print(f"ID: {experiment.experiment_id}, Name: {experiment.name}, Status: {experiment.lifecycle_stage}")


ID: 2, Name: test, Status: deleted
ID: 1, Name: nyc-taxi-expirement, Status: active
ID: 0, Name: Default, Status: active


In [20]:
from mlflow.tracking import MlflowClient

# Initialize the client
client = MlflowClient(tracking_uri="sqlite:///mlflow.db")

# List all experiments using the fluent API
experiments = client.search_experiments(view_type=ViewType.ALL)

# Print the list of experiments
for experiment in experiments:
    print(f"ID: {experiment.experiment_id}, Name: {experiment.name}, Status: {experiment.lifecycle_stage}")


ID: 2, Name: test, Status: deleted
ID: 1, Name: nyc-taxi-expirement, Status: active
ID: 0, Name: Default, Status: active


In [21]:
client.create_experiment(name="my-cool-experiment")

'3'

In [23]:
from mlflow.entities import ViewType

runs = client.search_runs(experiment_ids = '1',
                    filter_string = "",
                    run_view_type = ViewType.ACTIVE_ONLY,
                    max_results = 5,
                    order_by = ["metrics.rmse ASC"]
)

In [24]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 08b12a688db84694a3f14a34f59747b5, rmse: 6.3569
run id: c8462f1cccd143dbad205dec0f3fcfd3, rmse: 6.4039
run id: 1a4206615af9476ba4a6a81d9745a9c7, rmse: 6.4144
run id: 0562718ba82d4d089d9ccfbd0ce78911, rmse: 6.4942
run id: 733ced4edcc34866b16a7b283e73dc8f, rmse: 6.5007


In [25]:
# Register a model

import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")

In [28]:
run_id = "08b12a688db84694a3f14a34f59747b5"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri= model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '1' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1717443621262, current_stage='None', description=None, last_updated_timestamp=1717443621262, name='nyc-taxi-regressor', run_id='08b12a688db84694a3f14a34f59747b5', run_link=None, source=('/workspaces/mlops-zero-to-one/02-Experiment '
 'Tracking/mlruns/1/08b12a688db84694a3f14a34f59747b5/artifacts/model'), status='READY', status_message=None, tags={}, user_id=None, version=1>

In [29]:
model_uri

'runs:/08b12a688db84694a3f14a34f59747b5/model'

In [33]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version:{version.version}, aliases:{version.aliases}")

version:1, aliases:[]


C:\Users\aksha\AppData\Local\Temp\ipykernel_15068\2951549860.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [37]:
# Add aliases to the model version
aliases_to_add = ["production"]
model_version = 1
for alias in aliases_to_add:
    client.set_registered_model_alias(name=model_name, alias=alias, version=model_version)
print(f"Aliases {aliases_to_add} have been added to model {model_name} version {model_version}.")



Aliases ['production'] have been added to model nyc-taxi-regressor version 1.


In [38]:
# Define the tags to add or update
tags_to_add = {"stage": "production", "version_note": "latest stable version"}

for key, value in tags_to_add.items():
    client.set_model_version_tag(name=model_name, version=model_version, key=key, value=value)
print(f"Tags {tags_to_add} have been added to model {model_name} version {model_version}.")


Tags {'stage': 'production', 'version_note': 'latest stable version'} have been added to model nyc-taxi-regressor version 1.


In [39]:
# Set description for the model version
description = "This is the latest stable version of the model, now in production."
client.update_model_version(name=model_name, version=model_version, description=description)

print(f"Description for model {model_name} version {model_version} has been set.")

Description for model nyc-taxi-regressor version 1 has been set.


Comparing versions and selecting the new "Production" model
In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
Download the DictVectorizer that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
Preprocess the test set using the DictVectorizer so we can properly feed the regressors.
Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
Based on the results, update the "Production" model version accordingly.
Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.